# 利用YOLO,YOLOv2和最新的YOLOv3做实时物体检测
【翻译自】：https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088
**You only look once(YOLO)** 是一个用于实时物体识别的系统。在本文中我们将介绍YOLO，YOLOv2以及YOLO9000。对于只关心YOLOv3的人，可以直接调准到文章最后。下图展示的是YOLO网站提供的准确率和速度比较。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*ju1oaoIkVUkaIPAdpehlzA.png)
让我们从下图开始。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*EYFejGUjvjPcc4PZTwoufw.jpeg)
YOLO的目标检测结果如下：<br>
![avater](https://cdn-images-1.medium.com/max/800/1*QOGcvHbrDZiCqTG6THIQ_w.png)

### 网格单元
为了便于讨论，我们对原始图像进行裁剪。YOLO将输入图像分割为SxS个网格。**每个网格只预测一个物体**。例如，下图中黄色的网格尝试预测中心（蓝色点）落入网格单元的“人”这一类物体。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*6qZXYCDUkC5Bc8nRolT0Mw.jpeg)
每个网格单元预测固定数量的边界框。在本例中，黄色的网格预测了两个边界框（蓝色框）来定位“人”。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*4Y1PaY3ZgxKt5w84_0pNxw.jpeg)
然而，“只预测一个物体”这个规则却限制了识别物体的精度。因此，YOLO在精度上有一些欠缺。对于下图，YOLO只检测出了9个老人中的5个。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*j4PnWfxP3yoVPOFyI27tww.jpeg)
对于每个单元格来说，<br>
- 会预测B个边界框，每个框有一个置信得分
- 用B个边界框也只是检测一个物体
- 预测C个类别概率（C对应了物体的类别）
为了在PASCAL VOC上验证，YOLO使用了7x7的网格（SxS），2个边界框（B）以及20个类（C）。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*OuMJUWo2rXYA-GYU63NUGw.jpeg)
我们继续来看更多细节。每个边界框包含5个元素：(x,y,w,h)以及一个置信得分。这个置信得分反应的是框内包含物体的概率，以及边界框的准确率。我们边界框的宽度w和高度h按照图像的宽度和高度做标准化。x和y是对应单元格的偏移量。然而，x,y,w以及h都是0到1之间的数。每个单元包含20个条件化的类别概率。**条件化的类别概率**指的是检测到的物体归属于一个特定类别的概率（每个单元格上每个类别对应一个概率）。所以，YOLO的预测结果的尺寸是(S,S,Bx5+C)=(7,7,2x5+20)=(7,7,30)。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*jtnrhMFNwGxiQmkY6LkdCQ.jpeg)
YOLO主要的原理就是构建一个CNN网络来预测一个(7,7,30)的张量。他使用CNN网络将每个位置上的空间维度降到7x7x1024的维度。YOLO使用两个全连接层来做线性回归，得到了7x7x2个边界框预测结果（下面中间的图）。为了最终的预测，我们保留那些高置信得分（大于0.25）的框作为最终的预测结果（下面右图）。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*e0VY6U1_WMF2KBoKQNZvkQ.png)
每个预测框的置信得分计算如下：<br>
$$class\ confidence\ score=box\ confidence\ score \times conditional\ class\ probability$$
这个指标可以衡量分类和定位结果的置信度（一旦物体被检测到）。<br>
我们可以将以上得分和概率项轻易地汇总起来。下面是参考文献中数学定义。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*0IPktA65WxOBfP_ULQWcmw.png)

## 网络设计
![avater](https://cdn-images-1.medium.com/max/1000/1*9ER4GVUtQGVA2Y0skC9OQQ.png)
YOLO包含24个卷积层，以及两个全连接层。一些卷积层使用了1x1卷积层来减少特征图的深度。对于最后一个卷积层，它的输出张量是(7,7,1024)的。然后整个张量被扁平化。使用两个全连接层用作现行回归，它输出7x7x30个参数，然后重塑为(7,7,30)，即每个位置有两个边界框预测值。<br>
一个速度较快但是精度较低版本的YOLO，叫做Fast YOLO，只使用使用9个卷积层。<br>

## 损失函数
YOLO在每个单元格上预测多个边界框。为了计算相对于真值的损失，我们其实只需要一个检测到物体的边界框。为了达到这个目的，我们选择相对于真实值IoU（交并比）最高的那个。这使得最终的预测结果在特定的尺寸和纵横比上都达到了更好的效果。<br>
YOLO在预测结果和真实值之间使用平方和误差计算损失。损失函数包含如下：<br>
- 分类损失
- 定位损失（预测边界框和真实值的误差）
- 置信损失（边界框是否包含物体）

### 分类损失
如果检测到物体，每个单元格的分类损失就是每个类别概率之差的平方和：<br>
![avater](https://cdn-images-1.medium.com/max/800/1*lF6SCAVj5jMwLxs39SCogw.png)

### 定位损失
定位损失用于衡量预测边界框的位置和尺寸。我们只计算有物体边界框的损失。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*BwhGMvffFfqtND9413oiwA.png)
我们对不同尺寸大小框的绝对误差一视同仁，也就是说较大边框和较小边框的两像素误差是一样的。为了强调这一点，YOLO预测的是边界框长度和宽度的平方根的差之和。另外，为了强调边界框准确率的重要性，我们会在损失的基础上乘以一个$\lambda coord$（默认值是5）。

### 置信损失
如果框中检测到了物体，置信损失的表达式如下:<br>
![avater](https://cdn-images-1.medium.com/max/800/1*QT7mwEbyLJYIxTYtOWClFQ.png)
如果框中没有检测到物体，置信损失的表达式如下：<br>
![avater](https://cdn-images-1.medium.com/max/800/1*Yc_OJIXOoV2WaGQ6PqhTXA.png)
大多数的边界框都不包含物体。这会导致类别不平衡问题，也就是说我们训练模型检测到更多的是背景而不是物体。为了缓解这种情况，我们通过系数$\lambda nobj$（默认值0.5）对损失加权。

### 损失
最终的损失是把定位损失，置信损失和分类损失加起来。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*aW6htqx4Q7APLrSQg2eWDw.png)

## 推断：非极大值抑制(NMS)
YOLO会对同一个物体多次预测。为了修正这种情况，YOLO使用了NMS来去除低置信度的冗余预测。NMS会带来2-3%的mAP提升。<br>
这里是一种可能的NMS实现:<br>
1. 对预测结果按照置信得分排序。
2. 从最高的得分开始，无论位置，如果找到之前的预测跟当前预测有同样的类别，但是当前预测的IoU>0.5，就忽略掉该预测。
3. 重复第二步直到所有的预测结果被检查到。

## YOLO的好处
- 快，支持实时处理。
- 预测结果（物体位置和分类）是单一网络得到的，可以进行端到端地训练来提升准确率。
- YOLO的泛化能力较强。当从识别自然图像到其他领域如艺术图像时，YOLO的表现更好。
![avater](https://cdn-images-1.medium.com/max/800/1*XbOnbcZmc50hyhhTwhD5QA.png)\
- 候选区域方法将分类器限制在了特定的区域。YOLO在整张图像上做预测。由于更多的上下文，YOLO可以在背景区域得到更少的假正例。
- YOLO每个单元格只检测一个物体。这使得在预测时会又更多的空间多样性。

## YOLOv2
SSD是YOLO的一个有力的竞争对手，其在实时处理的基础上可以得到更高的准确率。相较于基于区域的检测器，YOLO定位方面的误差更高，召回率（定位所有物体结果的评价指标）更低。YOLOv2是YOLO的第二个版本，目标就是提高处理速度的同时提高准确率。

## 精度提升
### 批规范化
在卷积层加入批规范化可以省去Dropout，是的mAP得以提升2%。
### 高分辨率分类器
YOLO训练包含两个步骤。首先，我们训练一个如VGG16一样的网络。然后我们将全连接层替换为卷积层并且重新进行端到端的训练。YOLO使用224x224的图片来训练分类器，紧跟448x448的图片来做物体检测。YOLOv2从224x224的图片开始训练分类器然后用448x448的图片以更少的代数微调分类器。这使得检测器在训练时更容易收敛而且使得mAP提升4%。<br>
### 利用锚定边界框做卷积
如YOLO论文中提到的，早期的训练中对于不稳定的梯度是非常敏感的。因此初始情况下，YOLO会随机地假设一些边界框。这些假设对于一些物体的识别帮助很大，但是另外一些可能表现的不好，这会导致阶跃梯度变化。在早期训练时，预测结果往往会纠结选择什么样的形状进行匹配。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*7iwTsezrn-tSndx96twprA.jpeg)
在真实世界中，边界框并不是任意的。汽车有相似的形状，行人的横纵比都接近于0.41。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*krGqonOLMzSE_PWqH_LvQA.jpeg)
由于我们只需要一个正确的假设，因此如果我们在初始训练时选择差异性较大的，贴近于真实世界的边界框，训练过程会更加稳定。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*CGWTPTY0sfvQoxsS0X6VFg.jpeg)
例如，我们可以创建如下所示的5个候选框。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*8Q8r9ixjTiKLi1mrF36xCw.jpeg)
不同于选择任意的5个边界框，我们预测相对于以上如上图中5个边界框的偏移量。如果我们对偏移量的值做约束，我们可以保持预测结果的多样性并且使得每个预测关注在一个特定的形状。所以初始的训练过程会更稳定。<br>
在文章中，锚点也叫作**“先验”**。<br>
下面是我们对网络做出的改进:<br>
- 去掉用于预测边界框的全连接层。
![avater](https://cdn-images-1.medium.com/max/800/1*tavjieD0Bum_uX-svYUTKA.jpeg)
- 我们将类别预测从单元格级别改变为边界框级别。现在，每个预测包含边界框的4个参数，1个置信得分（是否包含对象）以及20个类别概率。也就是5个边界框每个包含25个参数，即每个单元格是125个参数。同YOLO一样，对象性预测仍然是预测真实值和候选边界框的IOU。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*UsqjfoW3sLkmyXKQ0Hyo8A.png)
- 为了生成7x7x125的预测结果，我们需要将最后一个卷积层替换为3x3x1024的卷积层。然后应用1x1卷积层将7x7x1024转换为7x7x125（详细可以参考DarkNet）。
![avater](https://cdn-images-1.medium.com/max/800/1*KiSd2CBfcs5af6oliHCqzw.jpeg)
- 将输入图像尺寸从448x448变更为416x416。这会导致一个奇数的空间尺寸(7x7 vs.8x8单元格)。一张图片的中心通常是由一个较大的物体占据的。如果是奇数单元格，可以更精确地定位物体。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*89qezEeLKJLpD8_fM_H4qQ.jpeg)
- 去掉一个池化层使得网络的输出为13x13，而不是7x7。<br>
锚定边界框使得mAP从69.5微微减小到了69.2，但是召回率从81%提高到了88%，即精度微微下降但是增加了检测到所有真实物体的机会。<br>
### 维度聚类
在很多领域中，边界框有很强的模式。例如，在自动驾驶中，两个最常见的边界框是不同距离上的车和行人。为了检测覆盖训练集最好的K个边界框，我们可以利用K-means在训练数据上做聚类，得到最好的K个聚类的中心。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*esSmI0UaMr-GrqkUGMg0hA.jpeg)
由于我们处理的是边界框而不是点，我们不能使用常规的空间距离来衡量数据点的距离。毫无疑问，我们使用IoU。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*l5wvrPjLlFp6Whgy0MqbKQ.png)
在左图中，我们绘制的不同的聚类数（锚点数）下锚点和真实值的平均IoU。随着锚点增加，精度平稳提升。为了得到最佳表现，YOLO选择了5个锚点。在右图中，是5个锚点的形状。紫蓝色的矩形是从COCO数据集中选择的，而黑色边框的矩形是从VOC2007数据集中选择的。这两种类型的边界框都是更加瘦高以贴近真实世界的边界框。<br>
下文中只要我们不对YOLO和YOLOv2对比，我们说YOLO其实就是说的YOLOv2。<br>
### 直接位置预测
我们预测的是相对于锚点的偏移量，但是如果我们不加限制的话，预测结果仍是随机的。YOLO预测了。